In [49]:
import codecs

# Change fp to tweets csv of social_spambots_2.csv from http://mib.projects.iit.cnr.it/dataset.html
csv_fp = '/home/david/Desktop/social_spambots_2.csv/tweets.csv'

# Open csv file and get the tweet part of the csv.
# Strip out newlines and quotes around text.
with codecs.open(csv_fp, 'r', encoding='utf-8', errors='ignore') as fdata:
    sentences = [x.split(',')[1].strip('\n').strip('"').lower()
                 if len(x.split(',')) > 1 else '' for x in fdata.readlines()]

In [50]:
# Tokenize sentences for gensim word2vec.
tokenizes_sentences = [sentence.split(' ') for sentence in sentences]    

In [65]:
print(len(tokenizes_sentences))
print(tokenizes_sentences[1])

428625


['mother', 'daughter', 'java', 'time', 'http://t.co/ipnflgy2da']

In [44]:
from gensim.models import Word2Vec
# Train a word2vec model.
model = Word2Vec(tokenizes_sentences, min_count=1)

In [68]:
list(model.wv.vocab)[:10]

['text',
 'mother',
 'daughter',
 'java',
 'time',
 'http://t.co/ipnflgy2da',
 '#nsfw',
 'xxx',
 'pics',
 'and']

In [67]:
model['dad']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-1.1923444 ,  0.03722963, -0.34402746,  1.6997719 , -1.353605  ,
        2.2645788 ,  1.7444781 ,  1.3539782 ,  2.706652  ,  0.23909087,
        2.9133337 , -1.7410809 ,  0.76657945, -2.1948323 , -0.87692946,
       -0.20004529, -0.13441263,  0.5593517 , -0.9452145 ,  0.07329213,
        0.08104166, -0.01589753, -0.09591292,  0.92735964, -1.459889  ,
        0.13852987,  0.62451535,  0.21375507,  0.5760842 , -1.6965506 ,
        1.9320047 , -0.8340434 , -0.7043022 ,  0.3261048 , -1.8425604 ,
       -0.78281015,  1.0364954 , -0.24991253,  1.67967   ,  1.1848933 ,
        1.4995235 ,  3.3195589 ,  0.22955664,  0.7151312 , -0.50204116,
       -0.513799  , -1.8790119 ,  0.16409817, -1.1665107 , -0.28429958,
       -0.37807912, -0.45824006,  0.95202947,  1.515169  ,  0.53270334,
        1.9701957 ,  0.34950376,  1.96296   ,  0.19186008, -1.0254179 ,
        0.47591653, -0.20876828,  0.06547433, -0.66618735,  2.4148076 ,
       -1.1586033 ,  0.3227014 , -0.24076381,  1.0078981 , -0.76

In [62]:
model.most_similar(positive=['dad'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('mom', 0.8802906274795532),
 ('brother', 0.8081018924713135),
 ('sister', 0.7884122133255005),
 ('phone', 0.7188187837600708),
 ('boyfriend', 0.7166639566421509),
 ('she', 0.7150614261627197),
 ('bro', 0.7042582035064697),
 ('mum', 0.6966201066970825),
 ('cousin', 0.69659423828125),
 ('son', 0.6924066543579102)]